# Segmentation client avec le clustering K-Means

La segmentation de la clientèle vous permet à une entreprise de personnaliser ses relations avec les clients, comme nous le faisons dans notre vie quotidienne.

Ce Notebook permet de préparer les données pour extraire des informations utiles pour le ciblage client. L'idée principale sera d'appliquer à une base de données de clients de commerce électronique un algorithme de  clustering K-means grace à la librairie scikit-learn. L'objectif final est d'obtenir un regroupement de nos clients de le cadre du campagne Agile Marketing. 

Ces explorations permettent de répondre à plusieurs questions:

**Cette base de données clients peut-elle être regroupée pour développer des relations personnalisées ?**

**Pour répondre à cette question 3 fonctionnalités seront créées et utilisées :** <br>
- produits commandés
- taux de retour moyen
- dépenses totales

**L'ensemble de données représente les données réelles des clients et des commandes entre novembre 2018 et avril 2019 et il est pseudonymisé pour des raisons de confidentialité.**


__NB: La partie clustring vous sera présentée après la fin de l'examen__


## Questions:
* [Q1](#Q1)
* [Q2](#Q2)
* [Q3](#Q3)
* [Q4](#Q4)
* [Q5](#Q5)
* [Q6](#Q6)
* [Q7](#Q7)
* [Q8](#Q8)
* [Q9](#Q9)
* [Q10](#Q10)
* [Q11](#Q11)
* [Q12](#Q12)
* [Q13](#Q13)

## Import de librairies

In [ ]:
# data wrangling
import pandas as pd
import numpy as np

# visualization
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# for data preprocessing and clustering
from sklearn.cluster import KMeans

%matplotlib inline
# to include graphs inline within the frontends next to code

%config InlineBackend.figure_format='retina'
#to enable retina (high resolution) plots

pd.options.mode.chained_assignment = None
# to bypass warnings in various dataframe assignments

## Charger les données dans un dataframe nommé customers_orders

__Q1 - Charger les données dans un dataframe nommé customers_roders__ <a class="anchor" id="Q1"></a>

In [ ]:
# load data into a dataframe


## Afficher les cinq premières lignes du fichier

__Q2 - Afficher les cinq premières lignes du fichier__ <a class="anchor" id="Q2"></a>

In [ ]:
# first rows of the dataset


## Afficher les informations (infos) relative au dataframe

__Q3 - Afficher les informations (infos) relative au dataframe__ <a class="anchor" id="Q3"></a>

In [ ]:
# first glance of customers_orders data


__Q3.a - Combien de colonnes sont de type `object` ?__

Merci de répondre dans la cellule ci-dessous

--------
Réponse:

_Ma réponse_

--------

## Affichez la liste complète des colonnes du dataframe

__Q4 - Affichez la liste complète des colonnes du dataframe__ <a class="anchor" id="Q4"></a>

## Investigation du résumé statistique (summary) des données du dataframe

#### Q5.a - Affichez le résumé statistique du dataframe (summary) <a class="anchor" id="Q5"></a>

In [ ]:
# descriptive statistics of the non-object columns


Il y a un nombre important de lignes dont `ordered_item_quantity` est 0 et `net_quantity` est inférieur à 0, ce qui signifie qu'elles ne sont pas du tout commandées/vendues ; mais le fait qu'ils aient des retours nécessite une petite investigation.

#### Q5.b - Calculez le nombre de lignes avec une valeur de `net quantity` négative  

In [ ]:
print("Number of rows that net quantity is negative:", ###)

#### Q5.c - Supprimer les lignes avec une valeur de `net quantity` négative

In [ ]:
# exclude not sold/ordered SKUs from the dataset


# 1. Produits commandés
Le nombre de produits commandés dans la colonne product_type par un client. <br>

## 1. Products ordered
It is the count of the products ordered in product_type column by a customer. <br>

**Ci-dessous des fonctions permettant pour identifier les clients qui commandent plusieurs produits**

In [ ]:
def encode_column(column):
    if column > 0:
        return 1
    if column <= 0:
        return 0


def aggregate_by_ordered_quantity(dataframe, column_list):
    '''this function:
    1. aggregates a given dataframe by column list, 
    as a result creates a aggregated dataframe by counting the ordered item quantities

    2. adds number_of_X ordered where X is the second element in the column_list 
    to the aggregated dataframe by encoding ordered items into 1

    3. creates final dataframe containing information about 
    how many of X are ordered, based on the first element passed in the column list'''

    aggregated_dataframe = (dataframe
                            .groupby(column_list)
                            .ordered_item_quantity.count()
                            .reset_index())

    aggregated_dataframe["products_ordered"] = (aggregated_dataframe
                                                 .ordered_item_quantity
                                                 .apply(encode_column))

    final_dataframe = (aggregated_dataframe
                       .groupby(column_list[0])
                       .products_ordered.sum() # aligned with the added column name
                       .reset_index())

    return final_dataframe

In [ ]:
# apply functions to customers_orders
customers = aggregate_by_ordered_quantity(customers_orders, ["customer_id", "product_type"])

__Q6 - Afficher les cinq première lignes du dataframe customers__ <a class="anchor" id="Q6"></a>

# 2. Average Return Rate
C'est le rapport entre la quantité d'articles retournés et la quantité d'articles commandés. Ce ratio est d'abord calculé par commande puis moyenné pour toutes les commandes d'un client.

In [ ]:
# aggregate data per customer_id and order_id, 
# to see ordered item sum and returned item sum
ordered_sum_by_customer_order = (customers_orders
                                 .groupby(["customer_id", "order_id"])
                                 .ordered_item_quantity.sum()
                                 .reset_index())

returned_sum_by_customer_order = (customers_orders
                                  .groupby(["customer_id", "order_id"])
                                  .returned_item_quantity.sum()
                                  .reset_index())

# merge two dataframes to be able to calculate unit return rate
ordered_returned_sums = pd.merge(ordered_sum_by_customer_order, returned_sum_by_customer_order)

In [ ]:
# calculate unit return rate per order and customer
ordered_returned_sums["average_return_rate"] = (-1 * 
                                             ordered_returned_sums["returned_item_quantity"] /
                                             ordered_returned_sums["ordered_item_quantity"])

__Q7 - Afficher les cinq dernières lignes du dataframe customers__ <a class="anchor" id="Q7"></a>

In [ ]:
# take average of the unit return rate for all orders of a customer
customer_return_rate = (ordered_returned_sums
                        .groupby("customer_id")
                        .average_return_rate
                        .mean()
                        .reset_index())

In [ ]:
return_rates = pd.DataFrame(customer_return_rate["average_return_rate"]
                            .value_counts()
                            .reset_index())

return_rates.rename(columns=
                    {"index": "average return rate",
                     "average_return_rate": "count of unit return rate"},
                    inplace=True)

__Q8 - Afficher les return_rates du plus grand `average return rate` au plus petit__ <a class="anchor" id="Q8"></a>

In [ ]:
# add average_return_rate to customers dataframe
customers = pd.merge(customers,
                     customer_return_rate,
                     on="customer_id")

# 3. Total spending
Total spending is the aggregated sum of total sales value which is the amount after the taxes and returns.

In [ ]:
# aggreagate total sales per customer id
customer_total_spending = (customers_orders
                           .groupby("customer_id")
                           .total_sales
                           .sum()
                           .reset_index())

__Q9 - Renommer la colonne `total_sales` par `total_spending` dans le dataframe customer_total_spending__ <a class="anchor" id="Q9"></a>

## Create features data frame

In [ ]:
# add total sales to customers dataframe
customers = customers.merge(customer_total_spending, 
                            on="customer_id")

__Q10 - Quel est le nombre de clients total__ <a class="anchor" id="Q10"></a>

In [1]:
print("The number of customers from the existing customer base:", ####)

SyntaxError: unexpected EOF while parsing (1240406984.py, line 1)

__Q11 - Supprimer la colonne `custoner_id` du dataframe__ <a class="anchor" id="Q11"></a>

In [2]:
# drop id column since it is not a feature


In [ ]:
customers.head()

### Visualize features

In [ ]:
fig = make_subplots(rows=3, cols=1,
                   subplot_titles=("Products Ordered", 
                                   "Average Return Rate", 
                                   "Total Spending"))

fig.append_trace(go.Histogram(x=customers.products_ordered),
                 row=1, col=1)

fig.append_trace(go.Histogram(x=customers.average_return_rate),
                 row=2, col=1)

fig.append_trace(go.Histogram(x=customers.total_spending),
                 row=3, col=1)

fig.update_layout(height=800, width=800,
                  title_text="Distribution of the Features")

fig.show()

## Scale Features: Log Transformation

In [ ]:
def apply_log1p_transformation(dataframe, column):
    '''This function takes a dataframe and a column in the string format
    then applies numpy log1p transformation to the column
    as a result returns log1p applied pandas series'''
    
    dataframe["log_" + column] = np.log1p(dataframe[column])
    return dataframe["log_" + column]

### 1. Products ordered

In [ ]:
apply_log1p_transformation(customers, "products_ordered")

### 2. Average return rate

In [ ]:
apply_log1p_transformation(customers, "average_return_rate")

### 3. Total spending

In [ ]:
apply_log1p_transformation(customers, "total_spending")

### Visualize log transformation applied features

In [ ]:
fig = make_subplots(rows=3, cols=1,
                   subplot_titles=("Products Ordered", 
                                   "Average Return Rate", 
                                   "Total Spending"))

fig.append_trace(go.Histogram(x=customers.log_products_ordered),
                 row=1, col=1)

fig.append_trace(go.Histogram(x=customers.log_average_return_rate),
                 row=2, col=1)

fig.append_trace(go.Histogram(x=customers.log_total_spending),
                 row=3, col=1)

fig.update_layout(height=800, width=800,
                  title_text="Distribution of the Features after Logarithm Transformation")

fig.show()

In [ ]:
customers.head()

__Q12 - Afficher uniquement les colonnes du dataframe contenant `log`__ <a class="anchor" id="Q12"></a>

In [ ]:
# features we are going to use as an input to the model


## Create K-means model

In [ ]:
# create initial K-means model
kmeans_model = KMeans(init='k-means++', 
                      max_iter=500, 
                      random_state=42)

In [ ]:
kmeans_model.fit(customers.iloc[:,3:])

# print the sum of distances from all examples to the center of the cluster
print("within-cluster sum-of-squares (inertia) of the model is:", kmeans_model.inertia_)

## Hyperparameter tuning: Find optimal number of clusters

In [ ]:
def make_list_of_K(K, dataframe):
    '''inputs: K as integer and dataframe
    apply k-means clustering to dataframe
    and make a list of inertia values against 1 to K (inclusive)
    return the inertia values list
    '''
    
    cluster_values = list(range(1, K+1))
    inertia_values=[]
    
    for c in cluster_values:
        model = KMeans(
            n_clusters = c, 
            init='k-means++', 
            max_iter=500, 
            random_state=42)
        model.fit(dataframe)
        inertia_values.append(model.inertia_)
    
    return inertia_values

### Visualize different K and models

In [ ]:
# save inertia values in a dataframe for k values between 1 to 15 
results = make_list_of_K(15, customers.iloc[:, 3:])

k_values_distances = pd.DataFrame({"clusters": list(range(1, 16)),
                                   "within cluster sum of squared distances": results})

In [ ]:
# visualization for the selection of number of segments
fig = go.Figure()

fig.add_trace(go.Scatter(x=k_values_distances["clusters"], 
                         y=k_values_distances["within cluster sum of squared distances"],
                         mode='lines+markers'))

fig.update_layout(xaxis = dict(
        tickmode = 'linear',
        tick0 = 1,
        dtick = 1),
                  title_text="Within Cluster Sum of Squared Distances VS K Values",
                  xaxis_title="K values",
                  yaxis_title="Cluster sum of squared distances")

fig.show()

## Update K-Means Clustering

In [ ]:
# create clustering model with optimal k=4
updated_kmeans_model = KMeans(n_clusters = 4, 
                              init='k-means++', 
                              max_iter=500, 
                              random_state=42)

updated_kmeans_model.fit_predict(customers.iloc[:,3:])

### Add cluster centers to the visualization

In [ ]:
# create cluster centers and actual data arrays
cluster_centers = updated_kmeans_model.cluster_centers_
actual_data = np.expm1(cluster_centers)
add_points = np.append(actual_data, cluster_centers, axis=1)
add_points

In [ ]:
# add labels to customers dataframe and add_points array
add_points = np.append(add_points, [[0], [1], [2], [3]], axis=1)
customers["clusters"] = updated_kmeans_model.labels_

In [ ]:
# create centers dataframe from add_points
centers_df = pd.DataFrame(data=add_points, columns=["products_ordered",
                                                    "average_return_rate",
                                                    "total_spending",
                                                    "log_products_ordered",
                                                    "log_average_return_rate",
                                                    "log_total_spending",
                                                    "clusters"])
centers_df.head()

In [ ]:
# align cluster centers of centers_df and customers
centers_df["clusters"] = centers_df["clusters"].astype("int")

In [ ]:
centers_df.head()

In [ ]:
customers.head()

In [ ]:
# differentiate between data points and cluster centers
customers["is_center"] = 0
centers_df["is_center"] = 1

# add dataframes together
customers = customers.append(centers_df, ignore_index=True)

In [ ]:
customers.tail()

### Visualize Customer Segmentation

__Q13 - Créer une colonne appelée `cluster_name` dont les valeurs sont la transformation des valeurs dans la colonne `clusters` en string__ <a class="anchor" id="Q13"></a>

*hint: astype, dtype, dtypes*

In [ ]:
# add clusters to the dataframe


In [ ]:
# visualize log_transformation customer segments with a 3D plot
fig = px.scatter_3d(customers,
                    x="log_products_ordered",
                    y="log_average_return_rate",
                    z="log_total_spending",
                    color='cluster_name',
                    hover_data=["products_ordered",
                                "average_return_rate",
                                "total_spending"],
                    category_orders = {"cluster_name": 
                                       ["0", "1", "2", "3"]},
                    symbol = "is_center"
                    )

fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()

## Check for Cluster Magnitude

In [ ]:
# values for log_transformation
cardinality_df = pd.DataFrame(
    customers.cluster_name.value_counts().reset_index())

cardinality_df.rename(columns={"index": "Customer Groups",
                               "cluster_name": "Customer Group Magnitude"},
                      inplace=True)

In [ ]:
cardinality_df

In [ ]:
fig = px.bar(cardinality_df, x="Customer Groups", 
             y="Customer Group Magnitude",
             color = "Customer Groups",
             category_orders = {"Customer Groups": ["0", "1", "2", "3"]})

fig.update_layout(xaxis = dict(
        tickmode = 'linear',
        tick0 = 1,
        dtick = 1),
                 yaxis = dict(
        tickmode = 'linear',
        tick0 = 1000,
        dtick = 1000))

fig.show()